Determine visit,ccd pairs where deconvolution happened
----------------------------------------------------------

We process the log messages of the following minimalistic ip_diffim run:

    (lsst-scipipe) [gkovacs@lsst-dev03 gkovacs]$ pwd
    /project/gkovacs
    
    (lsst-scipipe) [gkovacs@lsst-dev03 gkovacs]$ imageDifference.py /project/mrawls/hits2015/rerun/cw_processed5/ --output check_deconv_2019-06-18 
    --id filter=g -c doSelectSources=False -c doDetection=False -c doMeasurement=False -c doMerge=False -c doForcedMeasurement=False 
    -c doMatchSources=False -c doAddMetrics=False |& tee -a imgdiff_chk_deconv_2019-06-18.log

Visit, ccd numbers are identified by:

    imageDifference INFO: Processing {'filter': 'g', 'visit': 421606, 'ccdnum': 51}

``ip_diffim`` branch ``u/gkovacs/sp2019-06_false_positives`` prints the following additional log messages following the PSF warping:

    ip.diffim.subtractExposures INFO: visit_deconvolution_case
    # --- or ---
    ip.diffim.subtractExposures INFO: visit_normal_convolution_case
    
(The visit and ccd numbers are not part of the ExposureF objects, so we need to identify cases based on two log lines...)


In [43]:
import re

Regexps to match the start of a new image and the convolution, deconvolution markers later

In [44]:
exp_newvisit = re.compile("^imageDifference INFO:.+visit':\\s*([0-9]+),.+ccdnum':\\s*([0-9]+)}.*$")
exp_normal = re.compile("^ip.diffim.subtractExposures.+visit_normal_convolution_case$")
exp_deconv = re.compile("^ip.diffim.subtractExposures.+visit_deconvolution_case$")

In [45]:
# Test visit and ccdnum extraction
s="imageDifference INFO: Processing {'filter': 'g', 'visit': 410915, 'ccdnum': 51}"
m=exp_newvisit.match(s)
print (m.group(1),m.group(2))

410915 51


In [75]:
norm_list = []
deconv_list = []

In [91]:
# If (None,None) appears then matching is inconsistent...

Nprocessed = 0
status_seen = True

# /project/gkovacs/imgdiff_chk_deconv_2019-06-18.log
with open('imgdiff_chk_deconv_2019-06-18.log') as f:
    for line in f:
        line = line.strip()
        m = exp_newvisit.match(line)
        if m is not None:
            if not status_seen:
                print (f"Failed case {visit} {ccd}")
            visit = int(m.group(1))
            ccd = int(m.group(2))
            Nprocessed += 1
            status_seen = False
            continue

        m = exp_deconv.match(line)
        if m is not None:
            deconv_list.append((visit,ccd))
            visit = None
            ccd = None
            status_seen = True
            continue

        m = exp_normal.match(line)
        if m is not None:
            norm_list.append((visit,ccd))
            visit = None
            ccd = None
            status_seen = True
            continue
        
    

Failed case 412307 31
Failed case 412618 30


In [87]:
# Write out the deconvolution cases to a file

with open('deconvolution_visit_ccd.txt','w') as W:
    for visit, ccd in deconv_list:
        print(f"{visit:8d} {ccd:3d}",file=W)

In [88]:
print(len(norm_list), len(deconv_list), Nprocessed)

3519 1518 5039


In [90]:
print(deconv_list)

[(410929, 5), (410929, 6), (410929, 7), (410929, 9), (410929, 10), (410929, 11), (410929, 12), (410929, 13), (410929, 15), (410929, 16), (410929, 17), (410929, 20), (410929, 21), (410929, 22), (410929, 23), (410929, 26), (410929, 27), (410929, 28), (410929, 29), (410929, 30), (410929, 31), (410929, 33), (410929, 34), (410929, 35), (410929, 36), (410929, 37), (410929, 38), (410929, 40), (410929, 41), (410929, 42), (410929, 43), (410929, 44), (410929, 46), (410929, 47), (410929, 48), (410929, 49), (410929, 50), (410929, 52), (410929, 53), (410929, 54), (410929, 55), (410929, 56), (410929, 57), (410929, 58), (410929, 59), (410929, 60), (410929, 62), (410931, 5), (410931, 6), (410931, 7), (410931, 10), (410931, 11), (410931, 12), (410931, 15), (410931, 16), (410931, 17), (410931, 18), (410931, 20), (410931, 21), (410931, 22), (410931, 23), (410931, 24), (410931, 27), (410931, 28), (410931, 29), (410931, 30), (410931, 31), (410931, 34), (410931, 35), (410931, 36), (410931, 37), (410931, 38)